<a href="https://colab.research.google.com/github/Deep-of-Machine/Dacon_Movie_review/blob/main/%5BBaseline%5D_2_%ED%86%A0%ED%81%B0%ED%99%94%EC%99%80_%ED%92%88%EC%82%AC%ED%83%9C%EA%B9%85%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 두 번째 베이스라인

이전의 베이스라인은 잘 보셨나요?

Count Vectorizer을 중점적으로 다루어 보았었는데요

이번에는 자연어처리에서 빼놓을 수 없는 개념인

"토큰화(Tokenization)"와 "품사 태깅(POS Tagging)"에 대해 살펴보려 합니다.

자 시작해볼까요??

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# train.csv 학습/검증

### 필요한 패키지 불러오기

먼저 필요한 패키지들을 불러와줍니다.

In [2]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
#nltk.download('all') # 처음 실행 시 주석을 제거하고 실행해주세요!

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

### train 데이터 불러오기

In [3]:
data = pd.read_csv('/content/drive/MyDrive/dataset 2/train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [4]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

간단하게 결측치 여부를 확인해보았습니다.

결측치가 존재하지 않는 데이터셋이군요!

### 학습/검증셋 분리하기

모델링 후 성능을 검증하기 위해 학습셋과 검증셋으로 train 데이터를 나눠줍니다.

In [5]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [6]:
train # train 셋 확인

,index,id,document,label
0,3132,3133,진심 안보면 후회해요 꼭 보세요 진짜 달달함,1
1,1035,1036,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,1
2,3288,3289,-13000점 마이너스는업냐..,0
3,3712,3713,지루하다. 테이큰의 벽을 못 넘음,0
4,3724,3725,사건없이 설정과설명만 있을뿐/성유리 얼굴을 보니 돈이 좋구나,0
...,...,...,...,...
3745,4004,4005,"평점 쓰는 덴 줄 알았더니, 알바천국일세..",0
3746,3854,3855,긴장감도 잇고 감동도 잇는 최고의 작품^^,1
3747,3452,3453,엔젤의 캐릭터를 너무 잘 소화한 금성무. 동화같은 사랑이야기^^,1
3748,4235,4236,최고..... 말이 필요업슴 이젠 3보러갈꺼임..,1


### 데이터 정제하기

document 컬럼의 리뷰들을 전처리해줍니다. 현재 리뷰는 다양한 특수문자와 

가장 먼저 한글과 공백만 남긴 뒤 preprocessed라는 컬럼으로 저장하겠습니다.

In [7]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,3132,3133,진심 안보면 후회해요 꼭 보세요 진짜 달달함,1,진심 안보면 후회해요 꼭 보세요 진짜 달달함
1,1035,1036,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,1,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ
2,3288,3289,-13000점 마이너스는업냐..,0,점 마이너스는업냐
3,3712,3713,지루하다. 테이큰의 벽을 못 넘음,0,지루하다 테이큰의 벽을 못 넘음
4,3724,3725,사건없이 설정과설명만 있을뿐/성유리 얼굴을 보니 돈이 좋구나,0,사건없이 설정과설명만 있을뿐성유리 얼굴을 보니 돈이 좋구나


추가적으로 혹시 모를 다중공백도 제거해줍니다.

In [8]:
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,3132,3133,진심 안보면 후회해요 꼭 보세요 진짜 달달함,1,진심 안보면 후회해요 꼭 보세요 진짜 달달함
1,1035,1036,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,1,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ
2,3288,3289,-13000점 마이너스는업냐..,0,점 마이너스는업냐
3,3712,3713,지루하다. 테이큰의 벽을 못 넘음,0,지루하다 테이큰의 벽을 못 넘음
4,3724,3725,사건없이 설정과설명만 있을뿐/성유리 얼굴을 보니 돈이 좋구나,0,사건없이 설정과설명만 있을뿐성유리 얼굴을 보니 돈이 좋구나


### 토큰화

#### 토큰화란?

토큰화(Tokenization)란 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정을 의미합니다.

따라서 다음과 같이 크게 두 종류로 구분해볼 수 있습니다.

1. 문장 토큰화
2. 단어 토큰화

먼저 말뭉치를 문장 단위로 나누는 문장 토큰화(sentence tokenization)의 예시를 살펴보겠습니다.

In [9]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의합니다.

In [10]:
from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


따로 분리 기준으로 "!", "?", "." 등을 설정할 필요 없이 문장 단위로 말뭉치가 분리됩니다!

다음으로 단어 단위로 나누는 단어 토큰화(word tokenization)의 예시를 살펴보겠습니다.

In [11]:
from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


단어 단위로 깔끔하게 분리되네요. "!", "." 등의 구두점도 하나의 단어로 취급되는 것을 확인할 수 있습니다.

이렇게 토큰화에 대해서 간단히 살펴보았습니다. 

#### 한국어에서의 토큰화

한국어는 영어와 달리 형태소(뜻을 가진 말의 최소 단위)간의 결합으로 어절이 이루어집니다. ex) 철수 -가 책 -을 읽- -었- -다

따라서 한국어 전용 형태소 분석기가 많이 존재하는데요, 이번에는 koNLPy라는 패키지의 Okt라는 형태소 분석기를 사용해보겠습니다.

In [13]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 451 kB/s 
     |████████████████████████████████| 448 kB 42.1 MB/s 


In [14]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


stem=True 설정한 두 번째 결과를 보면 "해볼까" -> "해보다", "힘내서" -> "힘내다", "됩시다" -> "되다" 와 같이 각 형태소의 어간을 추출해준 것을 확인할 수 있습니다. 

필요에 따라서 선택적으로 사용하면 좋을 것 같네요!

이제 영화 리뷰 데이터에서 같은 방법으로 형태소 분석을 해볼까요?

In [15]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem
0,3132,3133,진심 안보면 후회해요 꼭 보세요 진짜 달달함,1,진심 안보면 후회해요 꼭 보세요 진짜 달달함,진심 안보 면 후회 하다 꼭 보다 진짜 달달 함
1,1035,1036,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,1,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,진짜 어리다 때 보다 지금 도 기 억 속 에 ㅋㅋㅋ
2,3288,3289,-13000점 마이너스는업냐..,0,점 마이너스는업냐,점 마이너스 는 업다
3,3712,3713,지루하다. 테이큰의 벽을 못 넘음,0,지루하다 테이큰의 벽을 못 넘음,지루하다 테이큰 의 벽 을 못 넘다
4,3724,3725,사건없이 설정과설명만 있을뿐/성유리 얼굴을 보니 돈이 좋구나,0,사건없이 설정과설명만 있을뿐성유리 얼굴을 보니 돈이 좋구나,사건 없이 설정 과 설명 만 있다 뿐 성유리 얼굴 을 보다 돈 이 좋다


형태소 분석 및 어간 추출을 완료한 "tokenized" 컬럼이 추가되었습니다!

### 품사 태깅(POS Tagging)

자 이제 다음 단계인 품사 태깅에 대해 알아봅시다.

품사 태깅이란 주어진 텍스트를 형태소 단위로 나눈 뒤, 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것입니다.

In [16]:
# 품사 태깅
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [17]:
print(okt.pos("이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다"))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


한 예시를 살펴보니 Noun, Josa, Adverb, Adjective, Verb, Suffix, Punctuation, KoreanParticle 등으로 품사가 태깅되어 하나의 리스트 형태로 반환됩니다

태그되는 품사의 종류는 더 다양하니 직접 다양한 문장을 이용해 공부해보세요!

okt의 pos 함수만을 이용해 토큰화와 품사 태깅을 한번에 할 수 있습니다.

품사 태깅은 꼭 필요한 품사(ex. 명사)를 추출할 때 유용하게 쓰일 수 있겠네요.

바로 명사를 추출해볼까요?

In [18]:
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


okt의 nouns 함수를 사용해 간편하게 명사만을 추출하였습니다!

이렇게 간단하게 명사만을 추출할수도 있지만, 영화 리뷰는 명사만으로 긍/부정을 판단하기 어렵기 때문에 

우선 임의로 명사, 동사, 형용사, 부사를 추출하여 사용해보겠습니다. (이 부분은 직접 고민하고 수정해보세요!)

In [19]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,3132,3133,진심 안보면 후회해요 꼭 보세요 진짜 달달함,1,진심 안보면 후회해요 꼭 보세요 진짜 달달함,진심 안보 면 후회 하다 꼭 보다 진짜 달달 함,진심 안보 후회 해요 꼭 보세요 진짜 달달 함
1,1035,1036,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,1,진짜 어렸을 때 봤는데 지금도 기억속에 ㅋㅋㅋ,진짜 어리다 때 보다 지금 도 기 억 속 에 ㅋㅋㅋ,진짜 어렸을 때 봤는데 지금 속
2,3288,3289,-13000점 마이너스는업냐..,0,점 마이너스는업냐,점 마이너스 는 업다,점 마이너스 업냐
3,3712,3713,지루하다. 테이큰의 벽을 못 넘음,0,지루하다 테이큰의 벽을 못 넘음,지루하다 테이큰 의 벽 을 못 넘다,지루하다 테이큰 벽 못 넘음
4,3724,3725,사건없이 설정과설명만 있을뿐/성유리 얼굴을 보니 돈이 좋구나,0,사건없이 설정과설명만 있을뿐성유리 얼굴을 보니 돈이 좋구나,사건 없이 설정 과 설명 만 있다 뿐 성유리 얼굴 을 보다 돈 이 좋다,사건 없이 설정 설명 있을 뿐 성유리 얼굴 보니 돈 좋구나


자 이제 총 세 개의 전처리된 컬럼이 생성되었습니다!

어떤 전처리 과정이 성능이 가장 잘 나올지는 미지수입니다. 직접 다양한 시도를 통해 최적의 전처리 프로세스를 찾아보세요~

이번 베이스라인에서는 마지막에 생성한 main_pos 컬럼을 이용해 모델 학습을 진행하겠습니다.

### 벡터화

CountVectorizer를 사용하였습니다. 

이 부분에 대한 설명은 첫 번째 베이스라인에 자세히 나와있으니 넘어가겠습니다.

In [20]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

### 모델 학습

In [22]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression()

### 검증셋으로 모델 성능 검증

먼저 검증 셋 val에 train 셋과 동일한 전처리 과정을 거쳐줍니다.

In [23]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,1823,1824,어이없는영화 죽은 왕따친구귀신은 도대채어디로간거 스토리가 이상해도 너무이상함,0,어이없는영화 죽은 왕따친구귀신은 도대채어디로간거 스토리가 이상해도 너무이상함,어이없다 영화 죽다 왕따 친구 귀신 은 도 대 채다 로간 거 스토리 가 이상하다 너...,어이없는 영화 죽은 왕따 친구 귀신 도 채어디 로간 거 스토리 이상해도 너무 이상함
1,2094,2095,캐릭터도 넘 귀엽고 해양생물에 대해 더 재미나게 알게 되여좋았어요,1,캐릭터도 넘 귀엽고 해양생물에 대해 더 재미나게 알게 되여좋았어요,캐릭터 도 넘다 귀엽다 해양 생물 에 대해 더 재미나 게 알 게 되다 좋다,캐릭터 넘 귀엽고 해양 생물 대해 더 재미나 알 되여 좋았어요
2,2612,2613,보지않는게 좋아요 재미도없고 안무섭고,0,보지않는게 좋아요 재미도없고 안무섭고,보지 않다 좋다 재미 도 없다 안 무섭다,보지 않는게 좋아요 재미 없고 무섭고
3,2601,2602,김소현 팬됬습니다....너무 재밌음,1,김소현 팬됬습니다너무 재밌음,김소현 팬 되다 너무 재밌다,김소현 팬 됬습니다 너무 재밌음
4,4906,4907,0점 주는이유 발연기가 별로임 목소리도 ㅎㅎ,0,점 주는이유 발연기가 별로임 목소리도 ㅎㅎ,점 주다 이유 발연기 가 별로 임 목소리 도 ㅎㅎ,점 주는 이유 발연기 별로 임 목소리


학습한 모델로 예측할 X를 벡터화해줍니다.

In [24]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [25]:
#run model
y_pred = model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

[0 1 0 ... 1 0 0]


In [26]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 정확도 확인

accuracy = 0.8088


train 셋으로 학습한 모델을 사용해 validation 셋을 예측해본 결과 0.8184의 정확도를 얻었습니다!

다양한 전처리 기법과 모델들을 사용하여 성능을 높여보세요~

# test.csv 분류하기

이제 본격적으로 대회에서 주어진 정답이 없는 test 데이터의 라벨을 예측해보겠습니다.

전처리 모델 학습 단계에서 test 데이터를 사용하면 data leakage로 간주되니

test.csv는 반드시 모델 학습이 끝난 뒤 예측시에만 등장해야 함에 주의해주세요!

train.csv를 이용한 전처리/학습을 test.cvs에 적용하는 것은 허용되지만, test.csv를 전처리/모델링에 직접적으로 이용하면 실격입니다.

In [28]:
test = pd.read_csv('/content/drive/MyDrive/dataset 2/test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


동일한 전처리 과정을 거치되, data leakage가 발생하지 않도록 주의합니다.

In [29]:
test.isna().sum()

id          0
document    0
dtype: int64

In [30]:
# preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [31]:
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 0]


# dacon 대회에 제출하기

이제 이 예측 결과를 submission.csv 파일로 만들어서 대회 페이지에 제출해보도록 합시다.

제출한 뒤 리더보드를 통해 결과를 확인합시다.

In [32]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('/content/drive/MyDrive/dataset 2/sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1


In [33]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['label'] = pred_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,0


In [34]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
submission.to_csv("submission_baseline2.csv", index=False)

이렇게 생성된 submission.csv 파일을 데이콘 대회 페이지에 업로드 & 제출하여 결과를 확인해보세요!

문제를 해결하기 위한 여러분의 방법을 코드 공유 게시판에 공유해주세요

좋아요와 댓글을 합산하여 가장 높은 점수를 얻으신 분께 데이콘 후드가 제공됩니다!